## Кластеризация (Clustering) - поиск взаимосвязанных сообщений

Имея набор обучающих образцов (training data items), которым уже сопоставлены классы, можно обучить модель и затем воспользоваться ей для классификации новых образцов (future data items). Мы назвали этот процесс __обучением с учителем (learning was guided by a teacher)__. Например, роль учителя может сводится к правильной классификации примеров.

Теперь допустим, что мы не располагаем метками (labels), с помощью которых можно было бы обучить модель классификации, например, потому что разметка обошлась бы слишком дорого. Что, если единственный способ получить миллионы меток - попросить, чтобы их вручную проставил человек? Можно попытаться найти какие-то __закономерности в самих данных__.

Рассмотрим __вопросно-ответный сайт (question and answer website)__. Когда пользователь будет искать на нашем сайте какую-то информацию, поисковая система, скорее всего, сразу покажет нужный ему ответ. Если имеющиеся ответы пользователя не устраивают, то сайт должен хотя бы показать близкие ответы (related answers), чтобы пользователь быстро понял, какие ответы существуют, и не ушел с сайта.

__Наивный подход__ - просто взять сообщение, вычислить его схожесть (similarity) со всеми остальными сообщениями и показать первые $n$ самых похожих сообщений в виде ссылок. Но очень скоро такое решение станет слишком накладным. Нужен метод, который быстро находит все взаимосвязанные сообщения (related posts).

Для достижения этой цели мы воспользуемся __кластеризацией__. Это метод такой организации данных, когда __похожие элементы (similar items) оказываются в одном кластере, а непохожие (dissimilar) - в разных__. 
* Первая проблема - как превратить текст в нечто такое, для чего можно вычислить сходство (similarity). 
* Располагая способом измерения похожести, можно с его помощью быстро построить кластер, содержащий похожие сообщения. 
* И останется только проверить, какие еще документы принадлежат этому кластеру. 

## Измерение сходства сообщений

С точки зрения машинного обучения "сырой" текст абсолютно бесполезен. Лишь преобразовав его в осмысленные числа, мы сможем подать их на вход алгоритмам машинного обучения, например, кластеризации. Это относится и к более приземленным операциям с текстом, в частности, измерению сходства.

### 1) Как не надо делать

Одной из мер сходства является __расстояние Левенштейна (Levenshtein distance)__, или __редакционное расстояние (Edit Distance)__. Пусть есть два слова: "machine" и "mchiene". Их сходство можно определить, как __минимальное число операций редактирования, необходимых для перехода от одного слова к другому__. В данном случае нужно всего две операции: добавить "a" после "m" и удалить первое "е". Однако это весьма дорогой алгоритм, потому что время его работы определяется произведением длин обоих слов.

Возвращаясь к сообщениям, мы могли бы схитрить: рассматривать слова целиком как символы и выполнять операции редактирования на уровне слов. Пусть есть два сообщения (для простоты ограничимся только заголовками): "How to format my hard disk" ("Как мне отформатировать жесткий диск") и "Hard disk format problems" ("Проблемы с форматированием жесткого диска"). Редакционное расстояние между ними равно 5, потому что нужно удалить слова "how", "to", "format", "my", а затем добавить в конец слова "format" и "problems". Следовательно, можно было бы определить различие между двумя сообщениями, как __количество слов, которые следует добавить или удалить, чтобы преобразовать одни текст в другой__. Эту идею можно было бы немного усовершенствовать, но по существу временная сложность остается той же самой.

Но даже если бы мы могли добиться достаточного быстродействия, существует еще одна проблема. В нашем примере слово "format" дает вклад 2 в расстояние, потому что мы сначала удалили его, а потом снова добавили. Следовательно, такое __расстояние неустойчиво (not robust) относительно изменения порядка слов__.

### 2) Как надо делать

Более надежное (robust) редакционное расстояние дает так называемый __набор слов (bag of word)__. При таком подходе порядок слов полностью игнорируется, а в основу кладутся просто счетчики вхождений слов. Каждому встречающемуся в сообщении слову сопоставляется количество его вхождений, и эти пары сохраняются в векторе. Неудивительно, что эта операция называется __векторизацией (vectorization)__. Обычно вектор получается очень большим, потому что содержит столько элементов, сколько есть слов во всем наборе данных. Возьмем, к примеру, два сообщения с такими счетчиками слов:

<img src = "img/two-example-posts.jpg" width=800>

Столбцы __"Occurrences in post 1"__ (Вхождений в сообщение 1) и __"Occurrences in post 2"__ можно рассматривать как простые векторы. Можно вычислить евклидово расстояние между вектором вопроса и векторами всех сообщений и взять, ближайшее сообщение (правда, как мы уже выяснили, это слишком медленно). А, кроме того, мы можем использовать их как векторы признаков на этапе кластеризации, применяя следующую процедуру:
1. Выделить характерные признаки из каждого сообщения и хранить его как вектор сообщений (vector per post).
2. Произвести кластеризацию этих векторов.
3. Определить кластер для сообщения (post) в вопросе.
4. Выбрать из этого кластера сколько-то сообщений, имеющих различное сходство с постом в вопросе. Это повышает
разнообразие (diversity).

## Количество общих слов как мера сходства (similarity measured)

### Преобразование простого текста в набор слов (bag of words)

Нам нет нужды писать свой код подсчета слов и представления набора слов в виде вектора. Метод __CountVectorizer__ из библиотеки SciKit не только умеет делать это эффективно, но и обладает очень удобным интерфейсом. Функции и классы из SciКit импортируются посредством пакета __sklearn__:

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)

Параметр __min_df (минимальная частота в документе)__ определяет, как CountVectorizer должен обходиться с редко встречающимися словами. Если его значением является __целое число__, то слова с меньшим числом вхождений, отбрасываются. Если значение - __дробное число__, то отбрасываются слова, доля которых во всем документе меньше этого числа. Параметр max_df интерпретируется аналогично. Распечатав объект, мы увидим все остальные параметры, которым SciKit присвоила значения по умолчанию:

In [2]:
print(vectorizer)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


Как и следовало ожидать, подсчитываются именно слова (__analyzer=word__), а что считать словом, определяется регулярным выражением __token_pattern__. Например, строка "cross-validated" будет разбита на два слова: "cross" и "validated". Пока не будем обращать внимания на прочие параметры и рассмотрим две строки из нашего примера, содержащие темы сообщений:

In [3]:
content = ["How to format my hard disk", " Hard disk format problems "]

Этот список строк можно подать на вход метода векторизатора __fit_transform()__, который и проделает всю работу:

In [4]:
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names()

[u'disk', u'format', u'hard', u'how', u'my', u'problems', u'to']

Векторизатор распознал семь слов, для каждого из которых мы можем получить счетчики:

In [5]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


__Это означает__, что первое предложение содержит все слова, кроме "problems", а второе - все слова, кроме "how", "my" и "to". На самом деле, это как раз те столбцы, которые присутствовали в предыдущей таблице. Из $X$ мы можем выделить вектор признаков, которым воспользуемся для сравнения документов.

Сначала __применим наивный подход__, чтобы показать проблемы, с которыми придется столкнуться на этапе предварительной обработки. Случайно выберем какое-то сообщение и создадим для него вектор счетчиков. Затем вычислим расстояния от него до всех векторов счетчиков и выберем сообщение, для которого расстояние минимально.

### Подсчет слов

Для экспериментов возьмем игрушечный набор данных, содержащий такие сообщения.

<img src = "img/toy-dataset-posts.jpg" width=700>

В этом наборе мы хотим найти сообщение, которое больше других похоже на сообщение "__imaging databases__"

Предполагаем, что сообщения находятся в каталоге __TOY_DIR__:

In [6]:
import os
import sys

TOY_DIR = "data/toy"
posts = [open(os.path.join(TOY_DIR, f)).read() for f in os.listdir(TOY_DIR)]
posts

['This is a toy post about machine learning. Actually, it contains not much interesting stuff.',
 'Imaging databases provide storage capabilities.',
 'Most imaging databases save images permanently.\n',
 'Imaging databases store data.',
 'Imaging databases store data. Imaging databases store data. Imaging databases store data.']

Задействуем для этой цели __CountVectorizer__:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)

Нам нужно уведомить векторизатор о полном наборе данных, чтобы он заранее знал, каких слов ожидать:

In [8]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

#samples: 5, #features: 25


Получилось __5 сообщений__ и __25 различных слов__ всё правильно. Подсчитаны следующие выделенные из текста слова:

In [9]:
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))

[u'about', u'actually', u'capabilities', u'contains', u'data', u'databases', u'images', u'imaging', u'interesting', u'is', u'it', u'learning', u'machine', u'most', u'much', u'not', u'permanently', u'post', u'provide', u'save', u'storage', u'store', u'stuff', u'this', u'toy']
25


Теперь векторизуем новое сообщение:

In [10]:
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

Отметим, что метод __transform__ возвращает разреженные векторы счетчиков, то есть в векторе не хранятся счетчики для каждого слова, потому что большая их часть равна нулю (в сообщении такое слово не встречается). Вместо этого используется потребляющая меньше памяти структура данных __coo_matrix__ (от слова "COOrdinate"). Так, для нашего сообщения вектор содержит всего два элемента:

In [11]:
print(new_post_vec)

  (0, 5)	1
  (0, 7)	1


Воспользовавшись методом __toarray()__, мы можем восстановить весь массив ndarray:

In [12]:
print(new_post_vec.toarray())

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


Весь массив нам понадобится, если мы захотим использовать вектор для вычисления сходства (similarity). __Чтобы измерить сходство__ (при наивном подходе), мы вычисляем евклидово расстояние между векторами счетчиков нового и всех старых сообщений:

In [13]:
import scipy as sp

def dist_raw(vl, v2):
    delta = vl - v2
    return sp.linalg.norm(delta.toarray())

Метод __norm()__ вычисляет евклидову норму (кратчайшее расстояние). Это самая очевидная метрика, но есть и много других определений расстояния.

Почитайте статью __"Distance Coefficients between Two Lists or Sets"__ на сайте __The Python Papers Sоurсе Codes__", где Марис Линь (Maurice Ling) описывает 35 разных расстояний.

Имея функцию __dist_raw__, мы теперь должны перебрать, все сообщения и запомнить самое близкое:

In [14]:
def find_best_post(X_train, new_post_vec, dist):
    best_dist = sys.maxint
    best_i = None

    for i in range(0, num_samples):
        post = posts[i]

        if post == new_post:
            continue

        post_vec = X_train.getrow(i)
        d = dist(post_vec, new_post_vec)

        print("=== Post %i with dist=%.2f: %s\n"%(i, d, post))

        if d < best_dist:
            best_dist = d
            best_i = i

    print ("Best post is %i with dist=%.2f"% (best_i, best_dist))

In [15]:
print("=== New Post: %s\n" % new_post)
find_best_post(X_train, new_post_vec, dist_raw)

=== New Post: imaging databases

=== Post 0 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.

=== Post 1 with dist=1.73: Imaging databases provide storage capabilities.

=== Post 2 with dist=2.00: Most imaging databases save images permanently.


=== Post 3 with dist=1.41: Imaging databases store data.

=== Post 4 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.

Best post is 3 with dist=1.41


* __Post 0__ сильнее всеrо отличается от __new_post__, т.к. в них нет ни одного общего слова.
* __Post 1__ очень похоже на __new_post__, но не является лучшим, т. к. содержит на одно отсутствующее в новом сообщение слово больше, чем __Post3__.
* __Post 4__ - это __Post 3__, повторенное трижды. Поэтому eгo сходство с новым сообщением должно быть точно таким же, как у __Post 3__.

Распечатка соответствующих векторов признаков объясняет, почему это не так:

In [16]:
print(X_train.getrow(3).toarray())
print(X_train.getrow(4).toarray())

[[0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[0 0 0 0 3 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0]]


Как видим, одних лишь счетчиков слов недостаточно. Необходимо нормировать векторы на единичную длину (unit length).

## Нормировка векторов счетчиков слов

В функции __dist_norm__ мы будем вычислять расстояние не между исходными, а между нормированными векторами:

In [17]:
def dist_norm(vl, v2):
    vl_normalized = vl / sp.linalg.norm(vl.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = vl_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

Тогда результаты измерения сходства изменятся следующим образом:

In [18]:
print("=== New Post: %s\n" % new_post)
find_best_post(X_train, new_post_vec, dist_norm)

=== New Post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.

=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.

=== Post 2 with dist=0.92: Most imaging databases save images permanently.


=== Post 3 with dist=0.77: Imaging databases store data.

=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.

Best post is 3 with dist=0.77


Теперь сходство Post 3 и 4 в точности одинаково. C точки зрения подсчета слов в сообщениях этот результат представляется правильным.

## Удаление малозначимых (less important) слов

Взглянем еще раз на Post 2. В нем встречаются следующие слова, отсутствующие в новом сообщении: "most" (большинство), "save" (сохранять), "images" (изображения) и "permanently" (постоянно). Но их значимость в сообщении совершенно различна. Слова типа "most", встречающиеся в самых разных контекстах, называются __стоп-словами (stop words)__. Они несут мало информации и потому должны весить меньше слов типа "images", которые встречаются отнюдь не во всех контекстах. Лучше всего вообще удалить слова, которые употребляются настолько широко, что не помогают выявить различия между текстами.

Этот шаг весьма типичен для обработки текста, поэтому в __CountVectorizer__ для него предусмотрен специальный параметр:

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1, stop_words='english')
X_train = vectorizer.fit_transform(posts)

Если вы точно знаете, какие стоп-слова хотели бы удалить, то можете передать их полный список. Если параметр __stop_words=english__, то список будет состоять из __318 английских слов__. Каких именно, покажет метод __get_stop_words()__:

In [20]:
print(sorted(vectorizer.get_stop_words())[0:20])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst']


__Новый список__ содержит на семь слов меньше:

In [21]:
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))

[u'actually', u'capabilities', u'contains', u'data', u'databases', u'images', u'imaging', u'interesting', u'learning', u'machine', u'permanently', u'post', u'provide', u'save', u'storage', u'store', u'stuff', u'toy']
18


__После исключения стоп-слов__ получаем такие результаты измерения сходства (similarity):

In [22]:
new_post_vec = vectorizer.transform([new_post])
print("=== New Post: %s\n" % new_post)

find_best_post(X_train, new_post_vec, dist_norm)

=== New Post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.

=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.

=== Post 2 with dist=0.86: Most imaging databases save images permanently.


=== Post 3 with dist=0.77: Imaging databases store data.

=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.

Best post is 3 with dist=0.77


__Теперь Post 1 и 2 сравнялись__. Но расстояния изменились несущественно, потому что наши демонстрационные сообщения очень короткие. Картина будет совершенно другой, если взять реальные тексты.

## Стемминг (Stemming)

Но одну вещь мы упустили. Одно и то же слово в разных грамматических формах мы считаем разными словами. Например, в Post 2 есть слова "imaging" и "images". Имеет смысл считать их, как одно слово, ведь они обозначают одно и то же понятие. Нам нужна функция, которая __производит стемминг, то есть выделяет из слова его основу__. В библиотеке SciКit стеммера нет. Но можно скачать бесплатную библиотеку __Natural Language Toolkit__ (NLTK), где имеется стеммер, который легко подключить к countVectorizer.

In [23]:
import nltk.stem

В NLTK есть несколько стеммеров. И это необходимо, потому что в каждом языке свои правила стемминга. Для английского языка возьмем класс __SnowballStemmer__.

In [24]:
s = nltk.stem.SnowballStemmer('english')
s.stem("graphics")

u'graphic'

In [25]:
s.stem("imaging")

u'imag'

In [26]:
s.stem("image")

u'imag'

In [27]:
s.stem("imagination")

u'imagin'

In [28]:
s.stem ( "imagine")

u'imagin'

Отметим, что результатом стемминга вовсе необязательно являются допустимые английские слова.

Стеммер работает и для глаголов:

In [29]:
s.stem("buys")

u'buy'

In [30]:
s.stem("buying")

u'buy'

В большинстве случаев, но не всегда:bought - форма прошедшего времени неправильного глагола buy(покупать). Как
видим, в этом случае стемммер ошибается.

In [31]:
s.stem("bought")

u'bought'

bought - форма прошедшего времени неправильного глагола buy(покупать). Как видим, __в этом случае стемммер ошибается__.

## Совместное использование векторизатора и стеммера из библиотеки NLTK

Нам нужно произвести стемминг сообщений перед их подачей классу __CountVectorizer__. В этом классе __есть несколько точек подключения__, позволяющих настроить этапы предварительной обработки и лексического анализа. __Препроцессор (preprocessor) и лексический анализатор (tokenizer)__ могут быть переданы конструктору в качестве параметров. Мы не хотим помещать стеммер ни туда, ни сюда, потому что тогда нам пришлось бы заниматься лексическим анализом и нормировкой самостоятельно. 

Вместо этого мы __переопределим метод build_analyzer__:

In [32]:
english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')
X_train = vectorizer.fit_transform(posts)

При этом каждое сообщение будет подвергнуто следующей обработке:
1. Сначала __на шаге предварительной обработки (preprocessing step)__ (в родительском классе) все буквы сообщения будут переведены в нижний регистр.
2. __На шаге лексического анализа (tokenization step)__ выделяются отдельные слова (в родительском классе).
3. И в завершение из каждого слова будет выделена основа.

В результате у нас получится на один признак меньше, потому что слова __"images" и "imaging" сольются__. Останется такой список признаков:

In [33]:
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))

[u'actual', u'capabl', u'contain', u'data', u'databas', u'imag', u'interest', u'learn', u'machin', u'perman', u'post', u'provid', u'save', u'storag', u'store', u'stuff', u'toy']
17


Если после объединения слов "images" и "imaging" прогнать новый векторнзатор со стеммингом для всех сообщений, то выяснится, что теперь __на новое сообщение больше всего похоже Post 2 поскольку оно дважды содержит понятие "imag"__:

In [34]:
new_post_vec = vectorizer.transform([new_post])
print("=== New Post: %s\n" % new_post)

find_best_post(X_train, new_post_vec, dist_norm)

=== New Post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.

=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.

=== Post 2 with dist=0.63: Most imaging databases save images permanently.


=== Post 3 with dist=0.77: Imaging databases store data.

=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.

Best post is 2 with dist=0.63


## Развитие концепции стоп-слов

Теперь, когда у нас есть разумный способ построить компактный вектор по зашумленному текстовому сообщению (noisy textual post), вернемся назад и подумаем, в чем на самом деле смысл значений признаков.

__Значения признаков (feature values)__ - это просто счетчики вхождения термов (terms) в сообщение. Мы предполагали, что __чем больше это значение, тем важнее терм для данного сообщения__. 

Но как быть, например, со словом "subject" (тема), которое естественно встречается в каждом сообщении? Можно, конечно, попросить countVectorizer удалить его, воспользовавшись параметром max_df. Например, если задать для него значение 0.9, то слова, встречающнеся в 90 и более процентах сообщений, будут игнорироваться. А если слово встречается в 89 процентах сообщений? __Как выбрать правильную величину max_df?__ Проблема в том, что какое бы значение ни выбрать, всегда какие-то термы будут важнее для различения (discriminative) документов, чем другие.

__Решить эту проблему можно только одним способом__ - подсчитав частоты термов для каждого сообщения и "оштрафовав" те, которые встречаются во многих сообщениях. Иными словами, мы хотим, чтобы данному терму в данном сообщении было сопоставлено большое значение, если он встречается в этом сообщении и мало где еще.

Именно в этом состоит смысл характеристики __"частота терма - обратная частота документа" (term frequency - inverse document frequency,__ или __TF-IDF)__. Здесь __TF__ относится к подсчету, а __IDF__ - к штрафу. Наивная реализация могла бы выглядеть так:

In [35]:
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log(float(len(D)) / (len([doc for doc in D if t in doc])))
    return tf * idf

__Мы не просто подсчитали термы, но и нормировали счетчики на длину документа__. Поэтому длинные документы не получат несправедливого преимущества перед короткими.

Если взять показанный ннже список __D__ уже разбитых на лексемы документов, то мы увидим, что __термы обрабатываются по-разному, хотя в каждом документе встречаются с одинаковой частотой__:

In [36]:
a, abb, abc = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [a, abb, abc]
print(tfidf("a", a, D))

0.0


In [37]:
print(tfidf("b", abb, D))

0.270310072072


In [38]:
print(tfidf("a", abc, D))

0.0


In [39]:
print(tfidf("b", abc, D))

0.135155036036


In [40]:
print(tfidf("c", abc, D))

0.366204096223


Видно, что __терм a__ не значим ни для одного документа, потому что встречается во всех. __Терм b__ важнее для документа __abb__, чем для __abc__, потому что встречается там дважды.

На практике граничных случаев (corner cases) больше, чем показано в этом примере. Но благодаря SciKit мы можем о них не думать, потому что все они учтены в классе __TfidfVectorizer__, наследующем CountVectorizer. Разумеется, не нужно забывать про наш стеммер:

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english', decode_error='ignore')

X_train = vectorizer.fit_transform(posts)

Теперь векторы документов вообще не содержат счетчиков. А __содержат они значения TF-IDF для каждого терма.__

In [42]:
new_post_vec = vectorizer.transform([new_post])
print("=== New Post: %s\n" % new_post)

find_best_post(X_train, new_post_vec, dist_norm)

=== New Post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.

=== Post 1 with dist=1.08: Imaging databases provide storage capabilities.

=== Post 2 with dist=0.86: Most imaging databases save images permanently.


=== Post 3 with dist=0.92: Imaging databases store data.

=== Post 4 with dist=0.92: Imaging databases store data. Imaging databases store data. Imaging databases store data.

Best post is 2 with dist=0.86


## Чего мы достигли и к чему стремимся

Пока что этап предварительной обработки (text pre-processing) включает следующие шаги:
1. Лексический анализ текста и разбиение его на лексемы (tokenizing).
2. Отбрасывание слов, которые встречаются слишком часто и потому не помогают находить релевантные сообщения.
3. Отбрасывание слов, которые встречаются так редко, что вряд ли встретятся в будущих сообщениях.
4. Подсчет оставшихся слов.
5. Вычисление TF-IDF по счетчикам с учетом всего корпуса текстов (text corpus).
Этот процесс позволяет преобразовать исходный зашумленный текст в компактное представление в виде значений признаков.

Но при всей простоте и эффективности подхода на основе набора слов с дополнительными расширениями у него имеется ряд недостатков, о которых следует знать.
* __Не учитываются связи между словами.__ Если принять описанный подход к векторизации, то у фраз "Car hits wall" (Машина врезалась в стену) и "Wall hits саr" (Стена врезалась в машину) будет один и тот же набор признаков.
* __Не улавливается отрицание.__ Например, фразы "I will eat ice cream" (Я стану есть мороженое) и "I will not eat ice сrеam" (Я не стану есть мороженое) с точки зрения векторов признаков очень похожи, но имеют противоположный смысл. Впрочем, эту проблему легко решить, если подсчитывать не только отдельные слова (__униграммы__), но также пары слов (__биграммы__) и тройки слов (__триграммы__).
* __Никак не обрабатываются ошибки в правописании__. Хотя человеку совершенно понятно, что слова "database" и "databas" означают одно и то же, в принятом подходе они считаются различными.